In [56]:
# regular expression opertation
import re
# string operation
import numpy
import string
# shuffle the list
from random import shuffle
# Linear algebra
import numpy as np
# data processing 
import pandas as pd
# NLTK library & download twitter dataset
import nltk
from nltk.corpus import twitter_samples
# module for stop words that come with NLTK
from nltk.corpus import stopwords
# module for stemming
from nltk.stem import PorterStemmer
# module for tokenizing strings
from nltk.tokenize import TweetTokenizer as tt
from sklearn.model_selection import train_test_split 
import sklearn.preprocessing as preproc
from sklearn.feature_extraction import text
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
#smart progressor meter
from tqdm import tqdm

nltk.download("twitter_samples")
# read the positive and negative tweets
pos_tweets = twitter_samples.strings("positive_tweets.json")
neg_tweets = twitter_samples.strings("negative_tweets.json")
print(f"positive sentment total samples {len(pos_tweets)}\nnegative sentment total samples {len(neg_tweets)}")



no_of_tweets = 2
print(f"Let's take a look at first {no_of_tweets} sample tweets:\n")
print("Example of Positive tweets:")
print("\n".join(pos_tweets[:no_of_tweets]))
print("\nExample of Negative tweets:")
print("\n".join(neg_tweets[:no_of_tweets]))
print("\nTweets may have URLs,numbers,and special characters.Hence,we need to process the text.")

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\a1248\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


positive sentment total samples 5000
negative sentment total samples 5000
Let's take a look at first 2 sample tweets:

Example of Positive tweets:
#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!

Example of Negative tweets:
hopeless for tmr :(
Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :(

Tweets may have URLs,numbers,and special characters.Hence,we need to process the text.


In [78]:
#Helper class for doing preprocrssing
class Twitter_Preprocess():
    def __init__(self):
        # instantiate tokenizer class
        self.tokenizer = tt(preserve_case=False,strip_handles=True,reduce_len=True)
        # get the EN stopwords
        self.stopwords = stopwords.words("english")
        # get the EN punctuation
        self.punctuation = string.punctuation
        # Instantiate stemmer object
        self.stemmer = PorterStemmer()
        
    def __remove_unwanted_characters__(self,tweet):
        
        # remove retweet style text "RT"
        tweet = re.sub(r'^RT[\s]+','',tweet)
        
        # remove hyperlink
        tweet = re.sub(r'http?:\/\/.*[\r\n]*','',tweet)
        
        # remove hashtag
        tweet = re.sub(r'#','',tweet)
        
        # remove email address
        tweet = re.sub(r'\S+@\S+','',tweet)
        
        # remove numbers
        tweet = re.sub(r'\d+','',tweet)
        
        # return removed text
        return tweet
    
    def __tokenize_tweet__(self,tweet):
        return self.tokenizer.tokenize(tweet)
    
    def __remove_stopwords__(self,tweet_tokens):
        
        tweets_clean = []
        
        for word in tweet_tokens:
            if (word not in self.stopwords and word not in self.punctuation):
                tweets_clean.append(word)
        return tweets_clean
    
    def __text_stemming__(self,tweet_tokens):
        # store the stemmed word
        tweets_stem = []
        
        for word in tweet_tokens:
            # stemming word
            stem_word = self.stemmer.stem(word)
            tweets_stem.append(stem_word)
        return tweets_stem # 注意 tab !! 
        
    def preprocess(self,tweets):
        tweets_processed = []
        for _,tweet in tqdm(enumerate(tweets)):
            # apply removing unwanted characters and remove style of retweet,URL
            tweet = self.__remove_unwanted_characters__(tweet)
            # apply nltk tokenizer
            tweet_tokens = self.__tokenize_tweet__(tweet)
            # apply stop word removal
            tweets_clean = self.__remove_stopwords__(tweet_tokens)
            # apply stemmer
            tweet_stems = self.__text_stemming__(tweets_clean)
            tweets_processed.extend([tweet_stems])
        return tweets_processed
  
    
    def build_bow_dict(self,tweets,labels):
        freq = {}
        for tweet,label in list(zip(tweets,labels)):
            for word in tweet : 
                freq[(word,label)] = freq.get((word,label),0) + 1
                
        return freq
    
    def extract_features(self,processed_tweet,bow_word_freq):
        # features array
        features = np.zeros((1,3))
        # bias term added in the 0th index
        features[0,0] = 1
        
        for word in processed_tweet:
            # get the positive freq of the word
            features[0,1] = bow_word_freq.get((word,1),0)
            # get the negative freq of the word
            features[0,2] = bow_word_freq.get((word,0),0)
            
        return features
           
    def sigmoid(self,z):
        
        h = 1 / (1+np.exp(-z))
        return h
    
    def predict_tweet(train_X,theta):
        
        y_pred = sigmoid(np.dot(train_X,theta))
        
        return y_pred
    
    def gradientDescent(train_X,train_Y,theta,J,y_pred,learning_rate=0.01,iterations=1000):
        
        m = len(train_Y)
        for i in range(iterations):
            
            error = y_pred - train_Y
            gradients = np.dot(train_X.T,error)
            theta = theta - (1/m)*learning_rate*gradients
     
        return theta
    
# initialize the text preprocessor class object
tp = Twitter_Preprocess()
# process tje positive and negative tweets
processed_pos_tweets = tp.preprocess(pos_tweets)
processed_neg_tweets = tp.preprocess(neg_tweets)
print(pos_tweets[:no_of_tweets])
print("=============================================")
print(processed_pos_tweets[:no_of_tweets])
print("=============================================")

labels = [1 for i in range(len(processed_pos_tweets))]
labels.extend([0 for i in range(len(processed_neg_tweets))])
    
twitter_processed_corpus = processed_pos_tweets + processed_neg_tweets
bow_word_freq = tp.build_bow_dict(twitter_processed_corpus, labels)

# shuffle the positive & negative tweets
shuffle(processed_pos_tweets)
shuffle(processed_neg_tweets)
# 建立標籤
positive_tweet_label = [1 for i in processed_pos_tweets]
negative_tweet_label = [0 for i in processed_neg_tweets]
# 建立資料架構
tweet_df = pd.DataFrame(list(zip( twitter_processed_corpus,positive_tweet_label+negative_tweet_label)),columns = ["processed_tweet","label"])
    
# train & test split
train_X_tweet,test_X_tweet,train_Y,test_Y = train_test_split(tweet_df["processed_tweet"],tweet_df["label"],test_size = 0.2,stratify=tweet_df["label"])
print(f"train_X_tweet {train_X_tweet.shape},test_X_tweet{test_X_tweet.shape},train_Y{train_Y.shape},test_Y{test_Y.shape}")
print("=============================================")
    
# train X feature dimension
train_X = np.zeros((len(train_X_tweet),3))
    
for index,tweet in enumerate(train_X_tweet):
    train_X[index, :] = tp.extract_features(tweet,bow_word_freq)
        
# test X feature dimension
test_X = np.zeros((len(test_X_tweet),3))
    
for index,tweet in enumerate(test_X_tweet):
        test_X[index, :] = tp.extract_features(tweet,bow_word_freq)
print(f"train_X {train_X_tweet.shape},test_X_tweet{test_X_tweet.shape}")
print("=============================================")
print(train_X[0:5])

# set the seed in numpy
np.random.seed(1)
# Apply the gradient descent of Logistic regression
# 0.1 as added L2 regularization
J,theta = tp.gradientDescent(train_X,np.array(train_Y).reshape(-1,1), np.zeros((3,1)), 1e-7)
    
print(f"The cost after traning is {J:.4f}.")
print(f"The resulting vector of weights is {[round(t,4) for t in np.squeeze(theta)]}")

predicted_probs = predict_tweet(test_X,theta)
predicted_labels = np.where(predicted_probs, 0.5, 1, 0)
print(f"Own implementation of logistic regression accuracy is {len(predicted_labels[predicted_labels == np.array(test_Y).reshape(-1,1)])/len(test_Y)*100:.2f}")
    
clf = LogisticRegression(random_state=42,penalty="12")
clf.fit(train_X,np.array(train_Y).reshape(-1,1))
y_pred = clf.predict(test_X)
y_pred_probs = clf.predict(test_X)
print(f"SKlearn logistic regression accuracy is {accuracy_score(test_Y , y_pred)*100:.2f}")

    

5000it [00:01, 4366.21it/s]
5000it [00:01, 4553.30it/s]


['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)', '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!']
[['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)'], ['hey', 'jame', 'odd', ':/', 'pleas', 'call', 'contact', 'centr', 'abl', 'assist', ':)', 'mani', 'thank']]
train_X_tweet (8000,),test_X_tweet(2000,),train_Y(8000,),test_Y(2000,)
train_X (8000,),test_X_tweet(2000,)
[[1.000e+00 6.200e+01 3.500e+01]
 [1.000e+00 6.330e+02 0.000e+00]
 [1.000e+00 3.582e+03 2.000e+00]
 [1.000e+00 3.582e+03 2.000e+00]
 [1.000e+00 3.582e+03 2.000e+00]]


AttributeError: 'Twitter_Preprocess' object has no attribute 'T'

In [77]:
s = '#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)'
re.sub(r'\d+', '', s) 
re.sub(r'#','',s)
re.sub(r'@','',s)
print(s)
print("==========================================================================================================")
print(type(train_X.T))
print(type(test_X))

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [76]:
x=np.array([1,2,3])
train_X = np.zeros((len(x),3))
print(x,"\n")
print(train_X,"\n")
y=np.array([1,1,1])
s=np.dot(x,y)
print(s)

[1 2 3] 

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] 

6
